## Libraries

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
import faiss
import json

2025-05-24 06:46:14.840053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748069175.030980      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748069175.087603      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Upload the model

In [ ]:
model = SentenceTransformer("embedding_model")

## Upload the data

In [ ]:
with open("/dataset.json", "r") as f:
    data = json.load(f)

In [45]:
len(data)

1603

In [1]:
def chunk_text(text, size=512, stride=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words) - size + 1, stride):
        chunk = " ".join(words[i:i+size])
        chunks.append(chunk)
    return chunks

In [2]:
all_chunks = []
all_metadata = []

In [ ]:
for doc_id, doc in enumerate(data):
    chunks = chunk_text(doc["content"])
    all_chunks.extend(chunks)
    all_metadata.extend([{"doc_id": doc['metadata']['document_id'], "chunk": chunk}] for chunk in chunks)

## Save metadata

In [44]:
with open("faiss_metadata.pkl", "wb") as f:
    pickle.dump(all_metadata, f)

## Embedding

In [46]:
chunk_embeddings = model.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/11797 [00:00<?, ?it/s]

## Save embedding result

In [ ]:
np.save("embeddings.npy", chunk_embeddings)

In [48]:
chunk_embeddings.shape

(377474, 384)

## Build and save FAISS index

IndexFlatL2 Index

In [ ]:
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

faiss.write_index(index, "faiss_index.idx")

IndexHNSWFlat Index

In [ ]:
dimension = chunk_embeddings.shape[1]
index = faiss.IndexHNSWFlat(dimension, 32)  # 32 = graph degree (tuneable)
index.hnsw.efConstruction = 40  # graph accuracy (higher = better recall)
index.add(chunk_embeddings)
faiss.write_index(index, "faiss_hnsw.index")